### 1.1 数据处理

In [56]:
import pandas as pd
import json

In [57]:
df = pd.read_csv("./口径相似度-数据.csv")

In [58]:
df.columns

Index(['IND_BUSINESS_DESC', 'OTHER_SYS_MODEL_DESC'], dtype='object')

In [59]:
df["OTHER_SYS_MODEL_DESC"] = df["OTHER_SYS_MODEL_DESC"].fillna("")

In [60]:
df["BUSINESS_SYS_DESC"] = df["IND_BUSINESS_DESC"] + df["OTHER_SYS_MODEL_DESC"]

In [61]:
df

,IND_BUSINESS_DESC,OTHER_SYS_MODEL_DESC,BUSINESS_SYS_DESC
0,统计贷款合同利息的合计值,SELECT SUM(CON_AMT*INTEREST_RATE)\n FROM UIMP...,统计贷款合同利息的合计值SELECT SUM(CON_AMT*INTEREST_RATE)\...
1,统计借记卡去重客户数,SELECT COUNT(DISTINCT CUST_ISN)\n FROM UIM...,统计借记卡去重客户数SELECT COUNT(DISTINCT CUST_ISN)\n ...
2,统计去重的贷款授信客户数,SELECT COUNT(DISTINCT CUST_ISN)\nFROM UIMP.F_A...,统计去重的贷款授信客户数SELECT COUNT(DISTINCT CUST_ISN)\nF...
3,统计信用卡分期待摊余额的合计值,SELECT SUM(MP_L_BAL) \n FROM UIM...,统计信用卡分期待摊余额的合计值SELECT SUM(MP_L_BAL) ...
4,统计信用卡授信额度的合计值,SELECT SUM(CRED_LMT)\n FROM UIMP.F_AC_CRDT_CA...,统计信用卡授信额度的合计值SELECT SUM(CRED_LMT)\n FROM UIMP...
...,...,...,...
457,按照机构维度分类，统计（1）+（2）的去重客户数：\n（1）贷款类型为“普通贷款”，昨日余额...,"SELECT\n A.SUPER_ORG_NO AS DM_000000_DIMORG,\...",按照机构维度分类，统计（1）+（2）的去重客户数：\n（1）贷款类型为“普通贷款”，昨日余额...
458,按照机构、客户维度分类，统计存款类型为“活期存款”，客户类型为“个人客户”，账户状态不为“销...,"SELECT \nA.SUPER_ORG_NO\n,B.CUST_ISN\n,SUM(B.A...",按照机构、客户维度分类，统计存款类型为“活期存款”，客户类型为“个人客户”，账户状态不为“销...
459,按照机构、客户维度分类，统计存款类型为“活期存款”，客户类型为“个人客户”，账户状态不为“销...,"SELECT\n A.BEL_ORG_NO AS DM_000000_DIMORG,\n ...",按照机构、客户维度分类，统计存款类型为“活期存款”，客户类型为“个人客户”，账户状态不为“销...
460,按照机构、客户维度分类，统计存款类型为“活期存款”，客户类型为“个人客户”，账户状态不为“销...,"SELECT\n A.BEL_ORG_NO AS DM_000000_DIMORG,\n ...",按照机构、客户维度分类，统计存款类型为“活期存款”，客户类型为“个人客户”，账户状态不为“销...


### 1.2 创建ES索引

In [3]:
from elasticsearch import Elasticsearch
# es = Elasticsearch("http://158.219.101.240:8200")
es = Elasticsearch("http://10.31.55.139:8105")

In [6]:
index_config = {
    "settings": {
        "number_of_shards": 1,
    },
    "mappings": {
        "properties": {
            "name": {
                "type": "text",
            },
            "nameVector": {
                "type": "dense_vector",
                "dims": 1024,
                "index": True,
                "similarity": "cosine",
            },
            "description": {
                "type": "text",
            },
            "descriptionVector": {
                "type": "dense_vector",
                "dims": 1024,
                "index": True,
                "similarity": "cosine",
            }
        }
    }
}

In [8]:
index_name = "griffinql_indicator1"
# es.indices.delete(index=index_name)
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, **index_config)
    print(f"Index: [{index_name}] create success!")

Index: [griffinql_indicator1] create success!


### 1.3 向量化接口

In [65]:
import requests
import json
import traceback
import time

In [66]:
VECTOR_URL = "http://158.219.101.240:8832/doc_vector/item_vector"
# VECTOR_URL = "http://10.31.55.139:8761/doc_vector/item_vector"
def gen_vector(text_list: list):
    start_time = time.time()
    try:
        resp = requests.post(
            url=VECTOR_URL,
            data=json.dumps({
                "serial_id": "123456",
                "source_list": text_list
            }),
            headers={"Content-Type": "application/json"}
        )
        vectors = resp.json()["data"]
    except Exception as err:
        print(f"Gen vector error {err}, traceback: {traceback.format_exc()}")
        vectors = []
    end_time = time.time()
    # print(f"Total time: {end_time - start_time}")
    return vectors

### 1.4 数据向量化入库

In [69]:
scenes = df.columns.to_list()
scenes

['IND_BUSINESS_DESC', 'OTHER_SYS_MODEL_DESC', 'BUSINESS_SYS_DESC']

In [73]:
from tqdm import tqdm
from typing import List, Dict
from elasticsearch.helpers import streaming_bulk

start_time = time.time()
for scene in scenes:
    batch_size = 32
    for batch_start in tqdm(range(0, len(df), batch_size), total=len(df) // batch_size):
        batch_end = batch_start + batch_size
        batch = df.iloc[batch_start:batch_end]
        docs = [
            {
                "subject": row[scene],
                "table_name": scene,
            } for index, row in batch.iterrows()
        ]

        def document_generator():
            """Documents generator.
            """
            doc_art_content_formated_list = [doc["subject"] for doc in docs]
            doc_art_content_formated_vectors = gen_vector(doc_art_content_formated_list)
            for doc, doc_art_content_formated_vector in zip(docs, doc_art_content_formated_vectors):
                doc["subject_vector"] = doc_art_content_formated_vector
                yield doc

        successes = 0
        for ok, action in streaming_bulk(client=es, index=index_name, actions=document_generator()):
            successes += ok
        if successes != len(docs):
            print(f"Failed to be added to ElasticSearch: {len(docs)-successes}/{len(docs)}")
end_time = time.time()
print(f"Total time: {end_time - start_time}")

15it [00:04,  3.75it/s]                        
15it [00:04,  3.47it/s]                        
15it [00:04,  3.59it/s]                        

Total time: 12.519971370697021
